In [2]:
from llama_index.core import PromptTemplate
import json
import torch
from llama_index.core import Document
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import StorageContext
from llama_index.core import VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.weaviate import WeaviateVectorStore
from pyvi import ViTokenizer
import weaviate
from weaviate.classes.init import Auth

/home/drissdo/Desktop/VN_LAW_QA_SYSTEM/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
WEAVIATE_URL="https://frqjst3xshkqyw7ayhy6a.c0.asia-southeast1.gcp.weaviate.cloud"
WEAVIATE_API_KEY="eQJokAVcxKpPbEXRjr5E2tSi1lvXen5JAsgT"
DATA_COLLECTION = "ND168"
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
MODEL_NAME = "dangvantuan/vietnamese-document-embedding"
CHUNK_SIZE = 512  # Optimized for Vietnamese text
CHUNK_OVERLAP = 50  # Small overlap to maintain context

In [4]:
# Setup vector store
client = weaviate.connect_to_weaviate_cloud(
    cluster_url=WEAVIATE_URL,
    auth_credentials=Auth.api_key(WEAVIATE_API_KEY),
)


# client = weaviate.connect_to_local(
#                      host="192.168.100.125",
#                         port=8080,
#                         grpc_port=50051
#                 )
vector_store = WeaviateVectorStore(
    weaviate_client=client,
    index_name=DATA_COLLECTION
)

In [5]:
index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store,
    embed_model = HuggingFaceEmbedding(model_name=MODEL_NAME, device=DEVICE, trust_remote_code=True,cache_folder="/home/drissdo/.cache/huggingface/hub"),
)

In [6]:
# Create retriever
retriever = index.as_retriever(
    vector_store_query_mode="hybrid",
    similarity_top_k=10,
    alpha=0.3,
)

In [7]:
question = "ABC"
retrieved_docs = retriever.retrieve(question)

for i, node in enumerate(retrieved_docs):
    print(f"Result {i+1}:\n")
    print(f"Text: {node.text}\n")
    print(f"Metadata: {node.metadata}\n")
    print(f"Score: {node.score}\n")
    print("="*50)

Result 1:

Text: loại phương_tiện : cá_nhân 
 mức phạt : phạt tiền từ 2.000.000 đồng đến 3.000.000 đồng 
 nội_dung vi_phạm : ( cá_nhân ) cố_ý thay_đổi , xóa dấu_vết hiện_trường vụ tai_nạn giao_thông , trừ các hành_vi vi_phạm_quy_định

Metadata: {'category': 'Cá nhân', 'fine_amount': 'Phạt tiền từ 2.000.000 đồng đến 3.000.000 đồng', 'violation_type': 'khác', 'original_text': 'Loại phương tiện: Cá nhân\nMức phạt: Phạt tiền từ 2.000.000 đồng đến 3.000.000 đồng\nNội dung vi phạm: (Cá nhân) Cố ý thay đổi, xóa dấu vết hiện trường vụ tai nạn giao thông, trừ các hành vi vi phạm quy định'}

Score: 0.30000001192092896

Result 2:

Text: loại phương_tiện : cá_nhân 
 mức phạt : phạt tiền từ 200.000 đồng đến 250.000 đồng 
 nội_dung vi_phạm : ( cá_nhân ) bán hàng rong hoặc bán hàng_hóa nhỏ_lẻ khác trên lòng_đường , trên vỉa_hè các tuyến phố có quy_định cấm bán hàng ; trừ các hành_vi vi_phạm_quy_định tại khoản 7 , khoản 9 điều này ; phơi thóc , lúa , rơm , rạ , nông , lâm , hải_sản trên đường_bộ ; đặt

In [8]:
# client.collections.delete_all()